In [1]:
import torch
import torch.nn as nn
import torchvision
from tensorboardX import SummaryWriter
import torchvision.utils as vutils
import numpy as np

#writer = SummaryWriter()
writer = SummaryWriter('runs/exp-2Test')

BATCH = 2
torch.cuda.empty_cache()

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

torch.cuda.get_device_name(0)

'NVIDIA GeForce 940MX'

In [3]:
class Block(nn.Module):
  def __init__(self, in_cn, out_cn, mid_cn = None):
    super(Block, self).__init__()
    if mid_cn is None:
      mid_cn = out_cn
    #cada block te dos convolucions
    self.conv1 = nn.Conv2d(in_cn, mid_cn, 3, padding=1)
    #self.bn1 = nn.BatchNorm2d(mid_cn)
    #definim la funció d'activació
    self.relu1 = nn.ReLU(inplace=True)

    self.conv2 = nn.Conv2d(mid_cn, out_cn, 3, padding=1)
    #self.bn2 = nn.BatchNorm2d(out_cn)
    #definim la funció d'activació
    self.relu2 = nn.ReLU(inplace=True)

  def forward(self, x):
    x = self.conv1(x)
    #x = self.bn1(x)
    x = self.relu1(x)
    x = self.conv2(x)
    #x = self.bn2(x)
    x = self.relu2(x)
    return x

  def layersList(self):
    return [self.conv1, self.bn1, self.conv2, self.bn2]

In [4]:
class Encoder(nn.Module):
  #a chs passem les dimensions dels Blocks que tindrem
  def __init__(self, in_cn, out_cn, pool=True):
    super(Encoder, self).__init__()
    self.pool = pool
    if pool:
      self.mp = nn.MaxPool2d(2)
    self.block = Block(in_cn, out_cn)

  def layers_list(self):
    return self.block.layersList()

  def forward(self, x):
    if self.pool:
      x = self.mp(x)
    x = self.block(x)
    return x

In [5]:
class Decoder(nn.Module):
  def __init__(self, in_cn, out_cn, mid_cn):
    super(Decoder, self).__init__()
    self.decoder = nn.Upsample(scale_factor=2, mode='nearest')
    self.block = Block(in_cn, out_cn, mid_cn=mid_cn)

  def layers_list(self):
    return self.block.layersList()

  def forward(self, x1, x2=None, x3=None):
    x1 = self.decoder(x1)
    if x2 is None and x3 is None:
      x = x1
    elif x3 is None:
      x = torch.cat([x2,x1], dim=1)
    else:
      x = torch.cat([x2,x3,x1], dim=1)
    x = self.block(x)
    return x

In [6]:
class Mid(nn.Module):
    def __init__(self, in_cn, out_cn, small_cn=None):
        super(Mid, self).__init__()
        self.mp = nn.MaxPool2d(2)
        self.conv1 = nn.Conv2d(in_cn, out_cn, 3, padding=1)
        if small_cn is None:
            self.conv2 = nn.Conv2d(out_cn, in_cn, 3, padding=1)
        else:
            self.conv2 = nn.Conv2d(out_cn, small_cn, 3, padding=1)

    def layers_list(self):
        return [self.conv1, self.conv2]
    
    def forward(self, x):
        x = self.mp(x)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


In [7]:
class OutConv(nn.Module):
    def __init__(self, in_cn, out_cn):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_cn, out_cn, 1)
    
    def layerslist(self):
        return [self.conv]

    def forward(self, x):
        x = self.conv(x)
        return x

In [8]:
class UNet(nn.Module):
  def __init__(self, num_class):
    super(UNet, self).__init__()
    self.enc1 = Encoder(3, 64, pool=False)
    self.enc2 = Encoder(64, 128)
    self.enc3 = Encoder(128, 256)
    self.enc4 = Encoder(256, 512)
    self.mid = Mid(512, 1024)
    self.dec1 = Decoder(1024, 256, 512)
    self.dec2 = Decoder(512, 128, 256)
    self.dec3 = Decoder(256, 64, 128)
    self.dec4 = Decoder(128, 64, 64)
    self.outcnv = OutConv(64, num_class)

  def forward(self, x):
    x1 = self.enc1(x)
    x2 = self.enc2(x1)
    x3 = self.enc3(x2)
    x4 = self.enc4(x3)
    x5 = self.mid(x4)
    x = self.dec1(x5, x4)
    x = self.dec2(x, x3)
    x = self.dec3(x, x2)
    x = self.dec4(x, x1)
    x = self.outcnv(x)

    return x

In [9]:
PATH = 'C:/Users/ger-m/Desktop/UNI/4t/TFG/minidataset/'

import numpy as np
from torch.utils.data import Dataset
import cv2


class MyDataset(Dataset):
    def __init__(self, lst):
        self.lst = lst
    
    def __getitem__(self, index):
        
        imsd = 'sd/' + self.lst[index]
        imhd = 'hd/' + self.lst[index]
        
        x = cv2.imread(PATH + imsd)
        y = cv2.imread(PATH + imhd)

        x = np.transpose(x,(2,0,1))
        y = np.transpose(y,(2,0,1))

        #return x, y
        return {'img':x, 'gth':y}

    def __len__(self):
        return len(self.lst)

In [10]:
f = open('C:/Users/ger-m/Desktop/UNI/4t/TFG/listfile.txt', 'r')
images = f.read()
images = images.split("\n")
f.close()

train = images[:int(len(images)*0.05)]
test = images[int(len(images)*0.05):int(len(images)*0.07)]

In [11]:
# Generators
training_set = MyDataset(train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=BATCH, shuffle=True)

validation_set = MyDataset(test)
validation_generator = torch.utils.data.DataLoader(validation_set, batch_size=BATCH, shuffle=True)

In [12]:
unet = UNet(num_class=3)
from torch import optim
criterion = nn.MSELoss()
optimizer = optim.Adam(unet.parameters(), lr=1e-4, weight_decay=1e-4)
unet.to(device)
print()

In [13]:
def train(epoch, dataloader, model, criterion, optimizer, dev):
    model.train()
    #print("despres train")
    lensamples = len(dataloader)
    #print("despres len")
    for i_batch, sample_batched in enumerate(dataloader):
        #print("abans images")
        images_ = sample_batched['img'].to(device = dev, dtype=torch.float)
        #print("abans ground")
        ground_ = sample_batched['gth'].to(device = dev, dtype=torch.float)
        n_iter = epoch*lensamples + i_batch
        
        #print(images_.shape)

        #print("abans out")
        output_ = model(images_)
        #print("despres out")
        if n_iter%100==0:
            xi = vutils.make_grid(images_, normalize=True, scale_each=True)
            xg = vutils.make_grid(ground_,  normalize=True, scale_each=True)
            xo = vutils.make_grid(output_, normalize=True, scale_each=True)
            x = torch.cat((xi,xg,xo),1)
            writer.add_image('train/output'+ str(n_iter) + '_' + str(i_batch), x, n_iter)
            print('imatge guardada')
        #print("abans loss")
        loss = criterion(output_, ground_)
        #print("despres loss")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print("xd")
        writer.add_scalar('train/loss', loss.item(), n_iter)
        
        print('Train -> sample/numSamples/epoch: {0}/{1}/{2}, Loss: {3}' \
              .format(i_batch, lensamples, epoch, loss.item()))

In [14]:
max_epochs = 5
for epoch in range(max_epochs):
    # Training
    train(epoch, training_generator, unet, criterion, optimizer, device)          

C:\Users\ger-m\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


imatge guardada
Train -> sample/numSamples/epoch: 0/1182/0, Loss: 10306.4970703125
Train -> sample/numSamples/epoch: 1/1182/0, Loss: 8977.8564453125
Train -> sample/numSamples/epoch: 2/1182/0, Loss: 6124.06396484375
Train -> sample/numSamples/epoch: 3/1182/0, Loss: 6289.6318359375
Train -> sample/numSamples/epoch: 4/1182/0, Loss: 8589.8740234375
Train -> sample/numSamples/epoch: 5/1182/0, Loss: 6795.96826171875
Train -> sample/numSamples/epoch: 6/1182/0, Loss: 7742.0087890625
Train -> sample/numSamples/epoch: 7/1182/0, Loss: 5282.04931640625
Train -> sample/numSamples/epoch: 8/1182/0, Loss: 5179.2099609375
Train -> sample/numSamples/epoch: 9/1182/0, Loss: 3178.016357421875
Train -> sample/numSamples/epoch: 10/1182/0, Loss: 3344.745361328125
Train -> sample/numSamples/epoch: 11/1182/0, Loss: 1920.812255859375
Train -> sample/numSamples/epoch: 12/1182/0, Loss: 2419.204345703125
Train -> sample/numSamples/epoch: 13/1182/0, Loss: 3743.239013671875
Train -> sample/numSamples/epoch: 14/1182/

In [18]:
torch.save(unet, 'model.pth')

In [19]:
torch.save(unet.state_dict(), 'model_weights.pth')

In [13]:
model = UNet(3)
model.load_state_dict(torch.load('model_weights.pth'))
model.to(device)
print()

In [14]:
def accuracy(im1, im2):
    i1 = im1.cpu().detach().numpy()
    i2 = im2.cpu().detach().numpy()
    res = cv2.absdiff(i1, i2)
    #res = res.astype(np.uint8)
    percentage = (np.count_nonzero(res) * 100)/ res.size
    return res#percentage

In [15]:
def validation(epoch, dataloader, model, criterion, dev):
    model.eval()
    total_losses = []
    lensamples = len(dataloader)
    for i_batch, sample_batched in enumerate(dataloader):
        images = sample_batched['img'].to(device=dev, dtype=torch.float)
        ground = sample_batched['gth'].to(device=dev, dtype=torch.float)
        n_iter = epoch*lensamples + i_batch
        
        output = model(images)

        if n_iter%100==0:
            xi = vutils.make_grid(images, normalize=True, scale_each=True)
            xl = vutils.make_grid(ground, normalize=True, scale_each=True)
            xo = vutils.make_grid(output, normalize=True, scale_each=True)
            x = torch.cat((xi,xl,xo),1)
            writer.add_image('validation/output', x, n_iter)
            
        loss = criterion(output, ground)

        writer.add_scalar('validation/accuracy', loss, n_iter)

        print('Validation -> sample/numSamples/epoch: {0}/{1}/{2}, Accuracy: {3}%' \
              .format(i_batch, lensamples, epoch, loss))

        total_losses.append(loss)
        del images
        del ground
        del n_iter
        del output
    mean_loss = torch.stack(total_losses).mean()
    print('Mean Accuracy: {}'.format(mean_loss))

In [16]:
EPOCH = 1

for epoch in range(EPOCH):
    validation(epoch, validation_generator, model, accuracy, device)


C:\Users\ger-m\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


AssertionError: scalar should be 0D